In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
# 导入常用库
import pandas as pd
import numpy as np

# 加载比赛数据
train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")

# 显示训练数据的形状和前几行
print("训练集维度:", train.shape)
print("测试集维度:", test.shape)

train.head()

# 查看所有列和数据类型
train.info()

# 检查缺失值数量
print("\n缺失值统计：")
print(train.isnull().sum())

# 数值特征的统计信息
train.describe()

# 类别特征的唯一值数量
categorical_columns = train.select_dtypes(include='object').columns
print("\n唯一值统计：")
for col in categorical_columns:
    print(f"{col}: {train[col].nunique()} 个唯一值")

训练集维度: (8693, 14)
测试集维度: (4277, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB

缺失值统计：
PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     18

In [3]:
# 1. 类别列：用众数填充
for col in ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']:
    train[col].fillna(train[col].mode()[0], inplace=True)
    test[col].fillna(test[col].mode()[0], inplace=True)

# 2. 数值列：Age 用中位数，其余消费类用 0
train['Age'].fillna(train['Age'].median(), inplace=True)
test['Age'].fillna(test['Age'].median(), inplace=True)

for col in ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']:
    train[col].fillna(0, inplace=True)
    test[col].fillna(0, inplace=True)

# 3. Name 和 Cabin 暂不处理

# 检查缺失值是否都填好了
print("训练集缺失值：")
print(train.isnull().sum())

训练集缺失值：
PassengerId       0
HomePlanet        0
CryoSleep         0
Cabin           199
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name            200
Transported       0
dtype: int64


/tmp/ipykernel_13/989515281.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train[col].fillna(train[col].mode()[0], inplace=True)
/tmp/ipykernel_13/989515281.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

In [4]:
# 先把 Cabin 分成三部分
train[['Deck', 'CabinNum', 'Side']] = train['Cabin'].str.split('/', expand=True)
test[['Deck', 'CabinNum', 'Side']] = test['Cabin'].str.split('/', expand=True)

# CabinNum 是数值型（部分模型可能用得上）
train['CabinNum'] = pd.to_numeric(train['CabinNum'], errors='coerce')
test['CabinNum'] = pd.to_numeric(test['CabinNum'], errors='coerce')

# 填充 Deck 和 Side 缺失值为众数
for col in ['Deck', 'Side']:
    train[col].fillna(train[col].mode()[0], inplace=True)
    test[col].fillna(test[col].mode()[0], inplace=True)

# 删除原始 Cabin 列
train.drop(columns=['Cabin'], inplace=True)
test.drop(columns=['Cabin'], inplace=True)

# 检查缺失值情况
print("训练集缺失值：")
print(train.isnull().sum())

训练集缺失值：
PassengerId       0
HomePlanet        0
CryoSleep         0
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name            200
Transported       0
Deck              0
CabinNum        199
Side              0
dtype: int64


/tmp/ipykernel_13/312136155.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train[col].fillna(train[col].mode()[0], inplace=True)
/tmp/ipykernel_13/312136155.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

In [5]:
# 将 CabinNum 转为数值（如果尚未转）
train['CabinNum'] = pd.to_numeric(train['CabinNum'], errors='coerce')
test['CabinNum'] = pd.to_numeric(test['CabinNum'], errors='coerce')

# 用训练集的中位数填补 CabinNum 的缺失值
cabin_median = train['CabinNum'].median()
train['CabinNum'].fillna(cabin_median, inplace=True)
test['CabinNum'].fillna(cabin_median, inplace=True)

# 检查是否完全没有缺失值
print("train 缺失值：\n", train.isnull().sum()[train.isnull().sum() > 0])
print("test 缺失值：\n", test.isnull().sum()[test.isnull().sum() > 0])


/tmp/ipykernel_13/1483713643.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['CabinNum'].fillna(cabin_median, inplace=True)


train 缺失值：
 Name    200
dtype: int64
test 缺失值：
 Name    94
dtype: int64


/tmp/ipykernel_13/1483713643.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test['CabinNum'].fillna(cabin_median, inplace=True)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

# 保存 PassengerId（用于提交）
test_ids = test['PassengerId'].copy()

# 删除不需要的列
train_model = train.drop(columns=['Name', 'PassengerId'], errors='ignore')
test_model = test.drop(columns=['Name', 'PassengerId'], errors='ignore')

# 特征和标签
X = train_model.drop(columns='Transported')
y = train_model['Transported']

# 拆分训练和验证集
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 数值和类别特征
num_cols = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'CabinNum']
cat_cols = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']

# 预处理器
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

# 模型管道
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# 训练模型
clf.fit(X_train, y_train)

# 验证精度
val_score = clf.score(X_val, y_val)
print(f"验证集 Accuracy：{val_score:.4f}")

# 预测测试集
preds = clf.predict(test_model)

# 生成提交文件
submission = pd.DataFrame({
    'PassengerId': test_ids,
    'Transported': preds.astype(bool)
})

submission.head()


验证集 Accuracy：0.8068


,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
